In [7]:
import csv
from collections import OrderedDict
import json


file_path = "../data/Global_Mobility_Report.csv"

In [13]:
# country ->
#   date -> (grocery_pharmacy, park, residential, retail_recreation, transit_station, workplace)

places = ["grocery_pharmacy", "park", "residential", "retail_recreation", "transit_station", "workplace"]

raw_data = OrderedDict()
dates = set()
# Open the file in read mode
with open(file_path, "r") as file:
    reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for r in reader:
        country = r[1]
        province = r[2]

        if country == "country_region":
            continue

        if province != "":
            continue

        date = r[8]
        dates.add(date)

        retail_recreation = float(r[9]) / 100 if r[9] != "" else 999.0
        grocery_pharmacy = float(r[10]) / 100 if r[10] != "" else 999.0
        park = float(r[11]) / 100 if r[11] != "" else 999.0
        transit_station = float(r[12]) / 100 if r[12] != "" else 999.0
        workplace = float(r[13]) / 100 if r[13] != "" else 999.0
        residential = float(r[14]) / 100 if r[14] != "" else 999.0

        # country
        if raw_data.get(country) == None:
            raw_data[country] = OrderedDict()
            for p in places:
                raw_data[country][p] = OrderedDict()
        
        # date
        for p in places:
            if raw_data[country][p].get(date) == None:
                raw_data[country][p][date] = []

        
        # places
        if grocery_pharmacy != 999.0:
            raw_data[country]["grocery_pharmacy"][date].append(grocery_pharmacy)
        if park != 999.0:
            raw_data[country]["park"][date].append(park)
        if residential != 999.0:
            raw_data[country]["residential"][date].append(residential)
        if retail_recreation != 999.0:
            raw_data[country]["retail_recreation"][date].append(retail_recreation)
        if transit_station != 999.0:
            raw_data[country]["transit_station"][date].append(transit_station)
        if workplace != 999.0:
            raw_data[country]["workplace"][date].append(workplace)

date_sorted = sorted(dates)

In [15]:
# average multiple values for each date
# fill in missing dates
mobility_data = OrderedDict()

for country in raw_data:
    mobility_data[country] = OrderedDict()
    for category in raw_data[country]:
        mobility_data[country][category] = OrderedDict()
        for date in raw_data[country][category]:
            if len(raw_data[country][category][date]) == 0:
                continue
            perc = sum(raw_data[country][category][date]) / len(raw_data[country][category][date])
            mobility_data[country][category][date] = perc
        
        # fill in missing dates 
        prev = 0.0
        for date in date_sorted:
            if mobility_data[country][category].get(date) == None:
                mobility_data[country][category][date] = prev
            prev = mobility_data[country][category][date]
        

In [8]:
# Convert the ordered map to a standard dictionary
standard_dict = dict(mobility_data)

# Save the standard dictionary as JSON
file_path = "../data/mobility_country_place_date.json"
with open(file_path, 'w') as file:
    json.dump(standard_dict, file)

In [8]:
f = open("../data/mobility_country_place_date.json")
data = json.load(f)
data
f.close()

In [6]:
mx = -1
mn = 1
ca = "grocery_pharmacy"
for country in data:
    for date in data[country][ca]:
        if data[country][ca][date] > mx:
            mx = data[country][ca][date]
        if data[country][ca][date] < mn:
            mn = data[country][ca][date]
print(mx,mn)

3.98 -1.0


In [9]:
dates = list(data['United Arab Emirates']['grocery_pharmacy'].keys())
countries = list(data.keys())


In [10]:
raw_data = OrderedDict()
m = 0
c = ''

with open("../../../JHU_COVID-19.csv", "r") as file:
    reader = csv.reader(file)
    for r in reader:
        country = r[0]
        province = r[1]
        if country not in countries:
            continue

        if province != "":
            continue
        date = r[4][:10]
        cases = int(r[6])
        long = r[7]
        if long == "":
            long = 0.0
        else:
            long = float(long)
        lat = r[8]
        if lat == "":
            lat = 0.0
        else:
            lat = float(lat)
        
        if country == "United States":
            long = -95.7
            lat = 37.1
        
        

        
            
        if raw_data.get(country) == None:
            raw_data[country] = OrderedDict()
            
        
        if date in dates:
            #print(date)
            raw_data[country][date] = [cases, long, lat]
            

for country in raw_data:
    prev = [0,0.0,0.0]
    for date in dates:
        if raw_data[country].get(date) == None:
            raw_data[country][date] = prev
        prev = raw_data[country][date]

for country in raw_data:
    for date in dates:
        if m < raw_data[country][date][0]:
            m = raw_data[country][date][0]

standard_dict = dict(raw_data)

# Save the standard dictionary as JSON
file_path = "../data/case_country_date.json"
with open(file_path, 'w') as file:
    json.dump(standard_dict, file, indent=4, sort_keys=True)
            
        

In [11]:
m

6399531

In [43]:
data = OrderedDict()
for date in dates:
    s = 0
    for c in countries:
        if c in raw_data.keys():
            s =  s + raw_data[c][date][0]
    data[date] = s

standard_dict = dict(data)

# Save the standard dictionary as JSON
file_path = "../data/case_date.json"
with open(file_path, 'w') as file:
    json.dump(standard_dict, file, indent=4, sort_keys=True)